In [1]:
!pip install iterative-stratification

In [2]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict, load_dataset
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
import ast
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from tqdm.auto import tqdm  # Для прогресс-бара


2025-05-05 18:01:00.578563: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746468060.750752      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746468060.801616      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
df = pd.read_csv('/kaggle/input/multi-l/cleared_new.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11346 entries, 0 to 11345
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  11346 non-null  int64 
 1   text        11346 non-null  object
 2   class       11346 non-null  object
dtypes: int64(1), object(2)
memory usage: 266.1+ KB


In [4]:
df['class'] = df['class'].apply(lambda x: ast.literal_eval(str(x)))

In [5]:
all_labels = ['личная жизнь', 'политика', 'реклама', 'соцсети', 'спорт', 'юмор']

# Сортируем метки для воспроизводимости
sorted_labels = sorted(all_labels)

# Инициализируем бинаризатор
mlb = MultiLabelBinarizer(classes=sorted_labels)
mlb.fit(sorted_labels)

binary_matrix = mlb.transform(df['class'])
df['labels'] = list(binary_matrix)
df.head()

,Unnamed: 0,text,class,labels
0,0,твой лучший секс спрятан делюсь каналом диплом...,"[реклама, личная жизнь]","[1, 0, 1, 0, 0, 0]"
1,1,кнопка start,[соцсети],"[0, 0, 0, 1, 0, 0]"
2,2,продолжение правильно моем сообществе вк ссылк...,[соцсети],"[0, 0, 0, 1, 0, 0]"
3,3,временем авторская телега уверенно тройке силь...,[соцсети],"[0, 0, 0, 1, 0, 0]"
4,4,двоюродная сестра сын антон двоюродный племянн...,[личная жизнь],"[1, 0, 0, 0, 0, 0]"


In [6]:
class_mapping = {label: idx for idx, label in enumerate(mlb.classes_)}
class_mapping

{'личная жизнь': 0,
 'политика': 1,
 'реклама': 2,
 'соцсети': 3,
 'спорт': 4,
 'юмор': 5}

In [7]:
pre_dataset = Dataset.from_pandas(df)
# Бинарные метки (n_samples, n_classes)
labels = np.array(pre_dataset["labels"])

# Стратифицированное разбиение
msss = MultilabelStratifiedShuffleSplit(
    n_splits=1,
    test_size=0.2,
    random_state=42
)

train_idx, val_idx = next(msss.split(np.zeros(len(labels)), labels))

train_dataset = pre_dataset.select(train_idx)
val_dataset = pre_dataset.select(val_idx)

In [8]:
def print_label_distribution(pre_dataset, name):
    labels = np.array(pre_dataset["labels"])
    label_counts = pd.DataFrame(labels).sum(axis=0)
    print(f"{name} распределение:\n{label_counts}")

print_label_distribution(train_dataset, "Train")
print_label_distribution(val_dataset, "Validation")

Train распределение:
0     851
1     622
2    1678
3    1144
4    2752
5    1745
dtype: int64
Validation распределение:
0    213
1    156
2    420
3    286
4    688
5    437
dtype: int64


In [9]:
df_train = train_dataset.to_pandas()
df_val = val_dataset.to_pandas()

print(df_train.shape)
print(df_val.shape)

(9076, 4)
(2270, 4)


In [10]:
type(df_train['text'][0])

str

In [11]:
# Токенизация
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

inputs = tokenizer(df_train['text'].to_list(), padding=True, truncation=True, return_tensors="pt")

# Создаем TensorDataset
dataset = TensorDataset(
    inputs['input_ids'],
    inputs['attention_mask'],
    torch.tensor(df_train['labels'], dtype=torch.float32)  # float32 для multi-label
)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

/tmp/ipykernel_31/909355386.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  torch.tensor(df_train['labels'], dtype=torch.float32)  # float32 для multi-label


In [12]:
# 4. DataLoader
batch_size = 16
dataloader = DataLoader(
    dataset,
    sampler=RandomSampler(dataset),
    batch_size=batch_size
)

In [13]:
# 5. Инициализация модели (остается без изменений)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained(
    'bert-base-multilingual-cased',
    num_labels=6,  # Автоматическое определение числа классов
    problem_type="multi_label_classification"
).to(device)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Функция вычисления accuracy
def compute_accuracy(preds, labels):
    if len(labels.shape) > 1:  # Для multi-label
        preds = (torch.sigmoid(preds) > 0.5).int()
    else:  # Для multi-class
        preds = torch.argmax(preds, dim=1)
    return accuracy_score(labels.cpu().numpy(), preds.cpu().numpy())

# Цикл обучения
epochs = 20
for epoch in range(epochs):
    model.train()
    total_loss = 0
    all_preds, all_labels = [], []
    
    # Прогресс-бар для обучения
    train_progress = tqdm(dataloader, desc=f'Epoch {epoch + 1}/{epochs} [Train]', leave=False)
    
    for batch in train_progress:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, labels = batch
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
        # Сбор статистики
        total_loss += loss.item()
        all_preds.append(outputs.logits.detach())
        all_labels.append(labels)
        
        # Обновление прогресс-бара
        train_progress.set_postfix({
            'loss': loss.item(),
            'lr': optimizer.param_groups[0]['lr']
        })
    
    # Вычисление accuracy
    epoch_preds = torch.cat(all_preds)
    epoch_labels = torch.cat(all_labels)
    acc = compute_accuracy(epoch_preds, epoch_labels)
    
    print(f"\nEpoch {epoch + 1} | Loss: {total_loss/len(dataloader):.4f} | Accuracy: {acc:.4f}")

Epoch 1/20 [Train]:   0%|          | 0/568 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
model.save_pretrained("/kaggle/working//my_bert_classifier")
tokenizer.save_pretrained("/kaggle/working//my_bert_classifier")

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np

def compute_metrics(preds, labels):
    # Для multi-class классификации
    if len(labels.shape) == 1:
        preds = np.argmax(preds, axis=1)
    # Для multi-label классификации
    else:
        preds = (preds > 0.5).astype(int)
    
    return {
        'f1': f1_score(labels, preds, average='macro'),
        'precision': precision_score(labels, preds, average='macro'),
        'recall': recall_score(labels, preds, average='macro'),
        'accuracy': accuracy_score(labels, preds)
    }

In [ ]:
# Токенизация
inputs_val = tokenizer(df_val['text'].to_list(), padding=True, truncation=True, return_tensors="pt")

# Создаем TensorDataset
dataset_val = TensorDataset(
    inputs_val['input_ids'],
    inputs_val['attention_mask'],
    torch.tensor(df_val['labels'], dtype=torch.float32)  # float32 для multi-label
)

# 4. DataLoader
batch_size = 16
test_loader = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=batch_size,
    shuffle=False
)

In [ ]:
from sklearn.metrics import f1_score
import numpy as np

model.eval()  # Переводим модель в режим оценки
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        
        # Для multi-class
        # preds = torch.argmax(logits, dim=1)
        
        # Для multi-label (раскомментировать):
        preds = (torch.sigmoid(logits) > 0.5).int()
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Расчет F1
f1 = f1_score(all_labels, all_preds, average='macro')
print(f"F1-score: {f1:.4f}")

In [ ]:
from sklearn.metrics import f1_score, hamming_loss

# Расчет F1
f1 = f1_score(all_labels, all_preds, average='macro')
print(f"F1-score macro: {f1:.4f}")
f1 = f1_score(all_labels, all_preds, average='micro')
print(f"F1-score micro: {f1:.4f}")
f1 = f1_score(all_labels, all_preds, average='weighted')
print(f"F1-score weighted: {f1:.4f}")
hamming = hamming_loss(all_labels, all_preds)
print("Hamming Loss:", hamming)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(
    all_labels,
    all_preds,
    target_names=sorted_labels  # Замените на свои названия классов
))

In [ ]:
model.save_pretrained("/kaggle/working//my_bert_classifier1")
tokenizer.save_pretrained("/kaggle/working//my_bert_classifier1")